## Imports

In [1]:
import jieba, json, math, os, re, sys, shutil, time
from datetime import datetime
import numpy as np
import tensorboardX

import torch, torchtext
import torch.nn as nn
import torch.nn.functional as F

## Functions

In [2]:
from fields import Field, Parms, Semantic, Vocab, _make_vocab
from utils import *

from nlp_db import nlp_db

from model_class import NLU_Classify

semantic = Semantic()
args = Parms()
vocab = Vocab(semantic)

args.manual_log = './manualog_transfromer1.log'
args.model_path = './model_stores/transformer_wiki1.pth'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

## Dialog Task

### Data Files

In [7]:
def pathFiles(rel_path):
    return [
        os.path.join(os.path.abspath(rel_path),
                     os.listdir(rel_path)[i])
        for i in range(len(os.listdir(rel_path)))
    ]

In [13]:
# get train, test file names
rel_path = "../dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/"
cfiles = pathFiles(rel_path)
[print(file) for file in cfiles]

botTrainFile = cfiles[1]
botTrainFile

/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/chatterbot.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/xiaohuangji.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/weibo.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/douban_single_turn.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/qingyun.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/vocab.txt
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/tieba.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/subtitle.tsv
/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/ptt.tsv


'/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/xiaohuangji.tsv'

### Vocab Preprocess

In [11]:
args.vocab_path = os.path.abspath(os.path.join(rel_path, 'vocab.txt'))

def _make_chatbot_vocab(file, vocab_path, thres = 2):
    word_dict = {}
    with open(file, "r", encoding='utf-8') as f:
        cnt = 0
        for l in f.readlines():
            for token in list(jieba.cut(l.strip().replace('\t',""))):
                if token not in word_dict:
                    word_dict[token] = 0
                else:
                    word_dict[token] += 1

    if not os.path.isfile(vocab_path):
        open(vocab_path,'a').close()

    with open(vocab_path, 'w') as f:
        for k, v in word_dict.items():
            if v > thres:
                print(k, file=f)

In [14]:
os.remove(args.vocab_path)
if not os.path.isfile(args.vocab_path):
    _make_chatbot_vocab(botTrainFile, args.vocab_path)

try:
    vocab.load(args.vocab_path)
except:
    print("Vocab not loaded")
vocab.size, vocab.__getitem__('吃'), vocab.__getitem__(
    '<pad>'), vocab.__getitem__('<unk>'), vocab.__getitem__('<sos>')

(32970, 527, 263, 266, 264)

### Data Process => Model Parms Get

In [15]:
def get_max_sent_len(file):
    with open(file, "r", encoding='utf-8') as f:
        maxlen, sent_count = 0, 0
        for l in f.readlines():
            maxlen = max([maxlen, max([len(sent) for sent in l.split()])])
            sent_count += 1
    
    return maxlen, sent_count

args.max_sent_len, args.sent_count = get_max_sent_len(botTrainFile)

In [16]:
args.batch_size = 1000
dirrm(args)

{'batch_size': 1000,
 'exts': ['.en.atok', '.de.atok'],
 'manual_log': './manualog_transfromer1.log',
 'max_dec_num': 50,
 'max_enc_num': 50,
 'max_sent_len': 242,
 'modes': ['train', 'val', 'test2016'],
 'n_sent': 5,
 'ndev': 1,
 'path': './data/multi30k/',
 'sent_count': 454129,
 'vocab_path': '/home/ubuntu/Studio/dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/vocab.txt'}

### Sentence pad

### Define the Model

[ref: Seq2Seq - Transformer](https://pytorch.org/tutorials/beginner/transformer_tutorial.html#define-the-model)

In [7]:
# def _generate_square_subsequent_mask(sz):
#     mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#     mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#     return mask

# _generate_square_subsequent_mask(3)

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [4]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

### batch data

In [5]:
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [6]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [7]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


### train define

In [8]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

### trainning

In [9]:
# Train
bptt = 35
best_val_loss = float("inf")
epochs = 10  # The number of epochs
best_model = None


if os.path.isfile(args.model_path):
    model.load_state_dict(torch.load(args.model_path))

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        torch.save(best_model.state_dict(), f = args.model_path)

    scheduler.step()

    test_loss = evaluate(best_model, test_data)
    print('=' * 89)
    print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(test_loss, math.exp(test_loss)))
    print('=' * 89)


/home/ubuntu/anaconda3/envs/geo/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   200/ 2981 batches | lr 5.00 | ms/batch 16.25 | loss  4.83 | ppl   124.74
| epoch   1 |   400/ 2981 batches | lr 5.00 | ms/batch 15.81 | loss  4.86 | ppl   129.21
| epoch   1 |   600/ 2981 batches | lr 5.00 | ms/batch 15.88 | loss  4.70 | ppl   110.18
| epoch   1 |   800/ 2981 batches | lr 5.00 | ms/batch 15.87 | loss  4.76 | ppl   116.74
| epoch   1 |  1000/ 2981 batches | lr 5.00 | ms/batch 15.90 | loss  4.75 | ppl   116.15
| epoch   1 |  1200/ 2981 batches | lr 5.00 | ms/batch 15.93 | loss  4.80 | ppl   121.55
| epoch   1 |  1400/ 2981 batches | lr 5.00 | ms/batch 15.92 | loss  4.83 | ppl   124.85
| epoch   1 |  1600/ 2981 batches | lr 5.00 | ms/batch 15.96 | loss  4.87 | ppl   130.16
| epoch   1 |  1800/ 2981 batches | lr 5.00 | ms/batch 15.98 | loss  4.85 | ppl   128.21
| epoch   1 |  2000/ 2981 batches | lr 5.00 | ms/batch 15.99 | loss  4.85 | ppl   127.33
| epoch   1 |  2200/ 2981 batches | lr 5.00 | ms/batch 16.01 | loss  4.73 | ppl   113.63
| epoch   1 |  2400/ 

| End of training | test loss  5.43 | test ppl   227.73
| epoch   6 |   200/ 2981 batches | lr 3.68 | ms/batch 16.32 | loss  4.62 | ppl   100.99
| epoch   6 |   400/ 2981 batches | lr 3.68 | ms/batch 16.23 | loss  4.65 | ppl   104.53
| epoch   6 |   600/ 2981 batches | lr 3.68 | ms/batch 16.22 | loss  4.48 | ppl    88.42
| epoch   6 |   800/ 2981 batches | lr 3.68 | ms/batch 16.24 | loss  4.55 | ppl    94.48
| epoch   6 |  1000/ 2981 batches | lr 3.68 | ms/batch 16.26 | loss  4.56 | ppl    95.80
| epoch   6 |  1200/ 2981 batches | lr 3.68 | ms/batch 16.25 | loss  4.59 | ppl    98.20
| epoch   6 |  1400/ 2981 batches | lr 3.68 | ms/batch 16.24 | loss  4.59 | ppl    98.82
| epoch   6 |  1600/ 2981 batches | lr 3.68 | ms/batch 16.26 | loss  4.65 | ppl   104.09
| epoch   6 |  1800/ 2981 batches | lr 3.68 | ms/batch 16.24 | loss  4.61 | ppl   100.80
| epoch   6 |  2000/ 2981 batches | lr 3.68 | ms/batch 16.25 | loss  4.62 | ppl   101.23
| epoch   6 |  2200/ 2981 batches | lr 3.68 | ms/batch

| epoch  10 |  2600/ 2981 batches | lr 2.99 | ms/batch 16.26 | loss  4.44 | ppl    85.00
| epoch  10 |  2800/ 2981 batches | lr 2.99 | ms/batch 16.24 | loss  4.41 | ppl    82.13
-----------------------------------------------------------------------------------------
| end of epoch  10 | time: 50.88s | valid loss  5.59 | valid ppl   268.52
-----------------------------------------------------------------------------------------
| End of training | test loss  5.51 | test ppl   246.87


### Prediction

T,R,K; Run initialization

In [11]:
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [6]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

predict_model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
predict_model.load_state_dict(torch.load(args.model_path))

<All keys matched successfully>

In [12]:
# Get Results  
bptt = 35
i = bptt * 2
# data, targets = get_batch(test_data, i)
data, targets = get_batch(train_data, i)
# best_model.eval()
output = predict_model(data)

In [ ]:
def data2sent(data, func = None):
    if func:
        return [[TEXT.vocab.itos[func(ind).data.item()] for ind in data[:, i]]
            for i in range(data.shape[1])]
    
    else:
        return [[TEXT.vocab.itos[ind.data.item()] for ind in data[:, i]]
            for i in range(data.shape[1])]
    
    
# data2sent(data)
data2sent(predict_model(data[1:10,:]), func = lambda word_tensor: torch.argmax(word_tensor, dim = -1))